In [4]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import json
import scipy
import librosa

In [5]:
def plot_waveforms(audio, fs):
    """Plots the waveform of audio in the time domain.
    
    Parameters:
        audio (numpy.ndarray): audio signal
        fs (int): sampling frequency (Hz) of audio signal
        
    """
    plt.figure(figsize=(12, 6))
    librosa.display.waveshow(audio, sr=fs, alpha=0.58)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()

In [6]:
def calculate_spectrum(audio, kind='mag'):
    """
    Calculates the spectrum of an audio signal.
    Parameters:
        audio (numpy.ndarray): audio signal
        kind (str): 'mag' for magnitude, 'phase' for phase, 'complex' for complex
    """
    spec = scipy.fft.fft(audio)
    if kind == 'mag':
        return 20*np.log10(np.abs(spec))
    elif kind == 'phase':
        return np.angle(spec)
    elif kind == 'complex':
        return 20*np.log10(spec)
    else:
        raise ValueError('Invalid kind')

In [ ]:
#Function to plot spectrum 
